In [1]:
import pandas as pd
from scipy.stats import qmc
import matplotlib.pyplot as plt
import jax
jax.config.update('jax_enable_x64', True)
import jax.numpy as jnp

from pathlib import Path

In [2]:
directory_path = Path.cwd()

In [3]:
load_dataset_from_csv_file = True

In [4]:
if not load_dataset_from_csv_file:
    doe_file_name = Path('doe.csv')
    length = len(pd.read_csv(directory_path / doe_file_name).index)
    relative_path_to_output_folder = Path('outputs/538735.hpc06.hpc/hyperelastic_data/')
    dataset = pd.DataFrame(columns=['F11', 'F12', 'F21', 'F22', 'P11', 'P12', 'P21', 'P22'])
    for i in range(length):
        experiment_folder = directory_path/relative_path_to_output_folder/ Path('point_{}'.format(i))
        result_file = experiment_folder/ Path('results.p')    
        try:
            result = pd.read_pickle(result_file)

            F = result['deformation_gradient'][-1]
            P = result['pk1_stress'][-1]

            dataset.loc[i] = [F[0,0], F[0,1], F[1,0], F[1,1], P[0,0], P[0,1], P[1,0], P[1,1]]

        except:
            print("No data")
        dataset.to_csv(directory_path / Path('hyperelastic_dataset.csv'), index=False)
else:
    dataset = pd.read_csv(directory_path / Path('hyperelastic_dataset.csv'))

In [5]:
dataset

,F11,F12,F21,F22,P11,P12,P21,P22
0,0.500000,0.000000,2.675503e-35,0.500000,-1.003314e+03,5.300205e-13,2.714273e-12,-1.003314e+03
1,0.000000,-2.250000,2.250000e+00,0.000000,-2.033929e-13,-4.419506e+03,4.419506e+03,2.611245e-13
2,2.209710,-0.972272,2.209710e+00,0.972272,1.679296e+03,-3.498076e+03,1.679296e+03,3.498076e+03
3,-0.972270,-2.209710,9.722720e-01,-2.209710,-3.498072e+03,-1.679294e+03,3.498072e+03,-1.679291e+03
4,-0.693610,-1.674530,1.674530e+00,-0.693610,-8.414588e+02,-2.031470e+03,2.031470e+03,-8.414588e+02
5,3.291320,-1.363310,1.363310e+00,3.291320,1.664354e+04,-6.893985e+03,6.893985e+03,1.664354e+04
6,-0.085720,-0.111837,3.205976e-01,0.418276,-7.442957e+25,5.704834e+25,-1.990062e+25,1.525329e+25
7,0.358766,-2.482930,8.661370e-01,1.028462,7.740370e+02,-8.977612e+02,1.868693e+03,3.718649e+02
8,-1.134030,-0.310925,2.255730e-01,-1.563130,-7.806819e+02,-1.320304e+02,1.552870e+02,-6.637612e+02
9,0.566981,-3.279500,2.850410e+00,0.652333,2.513611e+03,-1.101967e+04,1.263678e+04,2.191946e+03


In [6]:
def_grad = dataset[['F11', 'F12', 'F21', 'F22']].values.reshape(-1, 2, 2)
pk1_simulated = dataset[['P11', 'P12', 'P21', 'P22']].values.reshape(-1, 2, 2)

In [7]:
def get_continuum_quantities(deformation_gradient):
    C = jnp.matmul(deformation_gradient.T, deformation_gradient)
    I_1 = jnp.trace(C) + 1.0
    I_3 = jnp.linalg.det(C)
    J = jnp.sqrt(I_3)
    I_hat_1 = I_1 / J**(2/3)
    return I_hat_1, J

In [8]:
def strain_energy_density(deformation_gradient):
    I_hat_1, J = get_continuum_quantities(deformation_gradient)

    # material parameters
    mu = 180.5
    lmbda_m = 2.8
    D = 0.0025
    
    psi = (mu * ( 1/2 * (I_hat_1-3) +
                 1/(20 *lmbda_m**2.0) * (I_hat_1**2-9) +
                 11/(1050 *lmbda_m**4.0) * (I_hat_1**3-27) +
                 19/(7000 *lmbda_m**6.0) * (I_hat_1**4-81) +
                 519/(673750 *lmbda_m**8.0) * (I_hat_1**5-243) ) +
                 1/D * ((J**2-1)/2 - jnp.log(J))
    )

    return psi

In [9]:
def pk1_stress(deformation_gradient):

    strain_energy_density_gradient = jax.jacrev(strain_energy_density)
    
    pk1_stress = strain_energy_density_gradient(deformation_gradient)

    return pk1_stress

def pk2_stress(deformation_gradient, pk1_stress):
    return jnp.matmul(jnp.linalg.inv(deformation_gradient), pk1_stress)

def green_lagrange_strain(deformation_gradient):
    return 1/2 * (jnp.matmul(deformation_gradient.T, deformation_gradient) - jnp.eye(2))

In [10]:
def check_isclose(matrix1, matrix2):
    return jnp.allclose(matrix1, matrix2, atol=1e-4)

def check_symmetry(matrix):
    return jnp.allclose(matrix, matrix.T, atol=1e-4)

In [11]:
pk1_analytical = jax.vmap(pk1_stress)(def_grad)
pk2_analytical = jax.vmap(pk2_stress)(def_grad, pk1_analytical)
gl_strain = jax.vmap(green_lagrange_strain)(def_grad)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [12]:
pk2_simulated = jax.vmap(pk2_stress)(def_grad, pk1_simulated)

In [13]:
jax.vmap(check_isclose)(pk1_analytical, pk1_simulated)

Array([ True,  True,  True,  True,  True,  True, False,  True,  True,
        True], dtype=bool)

In [14]:
jax.vmap(check_isclose)(pk2_analytical, pk2_simulated)

Array([ True,  True,  True,  True,  True,  True, False,  True,  True,
        True], dtype=bool)

In [16]:
pk1_simulated[6]

array([[-7.44295654e+25,  5.70483418e+25],
       [-1.99006154e+25,  1.52532878e+25]])

In [17]:
pk1_analytical[6]

Array([[-7.95551888e+25,  6.09769996e+25],
       [-2.12710807e+25,  1.63037139e+25]], dtype=float64)

Here are the observations that I have regarding the RVE simulation:
 
1. For small "uniaxial load", we are able to retrieve analytical $\mathbf{P}$ results, and we obtained symmetric $\mathbf{S}$
2. For small symmetric deformation gradients, we are able to retrieve analytical $\mathbf{P}$ results, and we obtained symmetric $\mathbf{S}$
3. For slightly larger symmetric deformation gradients, we are able to retrieve analytical $\mathbf{P}$ results, however in one case we could not obtain symmetric $\mathbf{S}$. This could be because our 'symmetry tolerance' was set too high. Setting a slightly higher tolerance resulted in ~ symmetric $\mathbf{S}$
4. When tested with a slightly more extended set of experiments in the DoE, we obtained symmetric $\mathbf{S}$ (increased tolerance)
5. There was one case where we did not get a match for $\mathbf{P}$, and therefore naturally did not get symmetric $\mathbf{S}$. This experiment could not reach the $\mathbf{F}_\text{target}$, since the FE simulation lost its convergence during the displacement increments. I had done a "trick" in my code that we export the last converged step's strain and stress states as outputs, i.e. a $(\mathbf{\hat{F}}, \mathbf{\hat{P}})$ pair, so that we are not "wasting" any simulation runs. If we look carefully, what happens is that the $\mathbf{F}_\text{target}$ is in a 'rotated state', that is tricky to reach from $\mathbf{F}_\text{init} = \mathbf{1}$ when proceeding incrementally. The tricky aspect is that the RVE reaches an intermediate displacement state $\mathbf{F}_\text{crit}$ which has $\det \mathbf{F}_\text{crit}\to 0$. (Note that $\mathbf{F}_\text{target}$ by itself doesn't need to have any such problems). This throws the simulation off completely, and the RVE reach extremely high stress states (in that last converged step). Therefore in saved displacement state corresponding to $\mathbf{\hat{F}}$ , the error between the analytical stress $\mathbf{\hat{P}}_\text{analytical}$ and simulated stress $\mathbf{\hat{P}}_\text{RVE}$  gets magnified considerably. This is the reason we are getting a mismatch.